In [1]:
import csv
import pandas as pd

from collections import defaultdict
from datetime import datetime
from tqdm import tqdm

### Load data

Reading CSV file into a dictionary instead of using pd.read_csv() because indexing in pandas is very slow compared to standard Python dictionaries and I've found that it's much faster to compute segment features (length, duration, and speed) on dictionary due to inexpensive dictionary lookups.

In [2]:
d = defaultdict(lambda: defaultdict(dict))

def parse_bool(bool_str):
    if bool_str == "false":
        return False
    elif bool_str == "true":
        return True

with open("../data/data_scientist_case.csv", mode='r', encoding='utf-8-sig') as f:
    columns = f.readline().strip().replace("\"", "").split(",")

    for line in f:
        values = list(csv.reader([line], delimiter=',', quotechar='"'))[0]

        # driver_id
        d[values[1]][values[2]][columns[0]] = values[0]

        # segment_datetime
        d[values[1]][values[2]][columns[3]] = datetime.strptime(values[3], '%Y-%m-%d %H:%M:%S')

        # published_date
        d[values[1]][values[2]][columns[4]] = datetime.strptime(values[4], '%Y-%m-%d %H:%M:%S') # segment_datetime

        # signup_date
        d[values[1]][values[2]][columns[5]] = datetime.strptime(values[5], '%Y-%m-%d %H:%M:%S') # segment_datetime

        # fixed_signup_country
        d[values[1]][values[2]][columns[6]] = values[6].strip()

        # is_main_segment
        d[values[1]][values[2]][columns[7]] = parse_bool(values[7])

        # unit_seat_price_eur
        d[values[1]][values[2]][columns[8]] = float(values[8].replace(",", ""))

        # seat_offered_count
        d[values[1]][values[2]][columns[9]] = int(values[9])

        #seat_left_count
        d[values[1]][values[2]][columns[10]] = int(values[10])

        # confirmed_seat_count
        d[values[1]][values[2]][columns[11]] = int(values[11])

        # segment_distance_km
        d[values[1]][values[2]][columns[12]] = float(values[12].replace(",", ""))

        # from (lat, lon)
        d[values[1]][values[2]]["from"] = (float(values[13]), float(values[14]))

        # to (lat, lon)
        d[values[1]][values[2]]["to"] = (float(values[15]), float(values[16]))

        # is_comfort
        d[values[1]][values[2]][columns[17]] = parse_bool(values[17])

        # is_auto_accept_mode
        d[values[1]][values[2]][columns[18]] = parse_bool(values[18])

        # publication_site_id
        d[values[1]][values[2]][columns[19]] = values[19]

In [3]:
#Class to represent a graph
class Graph:
    def __init__(self):
        self.graph = defaultdict(set) #dictionary containing adjacency List
        self.vertices = set()
 
    # function to add an edge to graph
    def addEdge(self, u, v):
        self.vertices.add(u)
        self.vertices.add(v)
        self.graph[u].add(v)
 
    # A recursive function used by topologicalSort
    def topologicalSortUtil(self, v, visited, stack):
 
        # Mark the current node as visited.
        visited[v] = True
 
        # Recur for all the vertices adjacent to this vertex
        for i in self.graph[v]:
            if visited[i] == False:
                self.topologicalSortUtil(i, visited, stack)
 
        # Push current vertex to stack which stores result
        stack.insert(0, v)
 
    # The function to do Topological Sort. It uses recursive
    # topologicalSortUtil()
    def topologicalSort(self):
        # Mark all the vertices as not visited
        visited = defaultdict(bool)
        stack =[]
 
        # Call the recursive helper function to store Topological
        # Sort starting from all vertices one by one
        for v in self.vertices:
            if visited[v] == False:
                self.topologicalSortUtil(v, visited, stack)
 
        # Return contents of stack
        return stack

### Extracting graph features

Below is code for extracting segment length (number of subsegments), segment duration (if possible), and segment speed (if possible).

In [4]:
def compute_segment_features(trip_id):
    point_pair_to_segment = {}

    points = set([d[trip_id][x]["from"] for x in d[trip_id].keys()])
    points = points.union([d[trip_id][x]["to"] for x in d[trip_id].keys()])

    g = Graph()
    for segment_id in d[trip_id].keys():
        g.addEdge(d[trip_id][segment_id]["from"], d[trip_id][segment_id]["to"])
        point_pair_to_segment[
            d[trip_id][segment_id]["from"] + d[trip_id][segment_id]["to"]
        ] = segment_id

    # Verify that graph is correct
    # every node must be connected to all the nodes that come after it in sorted topological order
    sorted_locations = g.topologicalSort()
    for i in range(len(sorted_locations)):
        for j in range(i + 1, len(sorted_locations)):
            if sorted_locations[j] not in g.graph[sorted_locations[i]]:
                for segment in d[trip_id].keys():
                    d[trip_id][segment]["length"] = None
                    d[trip_id][segment]["duration"] = None
                    d[trip_id][segment]["speed"] = None
                return

    seg_to_seg = defaultdict(list)
    for i in range(len(sorted_locations)):
        cur_segments = []
        for j in range(i + 1, len(sorted_locations)):
            cur_segments.append(
                point_pair_to_segment[(sorted_locations[j - 1] + sorted_locations[j])]
            )
            seg_to_seg[
                point_pair_to_segment[(sorted_locations[i] + sorted_locations[j])]
            ] = [x for x in cur_segments]

    sorted_core_segments = []
    for i in range(len(sorted_locations) - 1):
        sorted_core_segments.append(
            point_pair_to_segment[sorted_locations[i] + sorted_locations[i + 1]]
        )

    segment_durations = {}
    for i in range(len(sorted_core_segments) - 1):
        s1 = sorted_core_segments[i]
        s2 = sorted_core_segments[i + 1]
        duration = (
            d[trip_id][s2]["segment_datetime"] - d[trip_id][s1]["segment_datetime"]
        )
        duration = duration.seconds / 3600
        if duration != 0:
            segment_durations[s1] = duration
        else:
            segment_durations[s1] = None

    for segment in d[trip_id].keys():
        if segment not in segment_durations:
            subsegs = seg_to_seg[segment]
            s = 0
            all_known = True
            for seg in subsegs:
                if seg in segment_durations and segment_durations[seg] != None:
                    s += segment_durations[seg]
                else:
                    all_known = False
                    break
            if all_known:
                segment_durations[segment] = s

    segment_speeds = {}
    for segment in segment_durations:
        duration = segment_durations[segment]
        if duration != None and duration > 0:
            segment_speeds[segment] = (
                d[trip_id][segment]["segment_distance_km"] / segment_durations[segment]
            )
        else:
            segment_speeds[segment] = None

    segment_lengths = {}
    for segment in seg_to_seg:
        segment_lengths[segment] = len(seg_to_seg[segment])

    for segment in segment_lengths:
        d[trip_id][segment]["length"] = segment_lengths[segment]
        d[trip_id][segment]["duration"] = segment_durations.get(segment)
        d[trip_id][segment]["speed"] = segment_speeds.get(segment)

In [5]:
for trip_id in tqdm(d.keys()):
    compute_segment_features(trip_id)

100%|██████████████████████████████████████████████████████| 704411/704411 [00:21<00:00, 32548.06it/s]


In [6]:
df = pd.DataFrame.from_dict({
    (i, j): d[i][j] for i in d.keys() for j in d[i].keys()
}, orient="index")

In [7]:
df.index.names = ["trip_id", "segment_id"]
df.to_csv("../data/with_segment_features.csv")